# speed rush to overview torch.

In [3]:
import torch
import torchvision
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(device)


mps


In [ ]:
train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor()
)

